In [1]:
from ib_insync import *
import pandas as pd
from datetime import datetime, timedelta
import requests
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
import random

In [2]:
util.startLoop()

In [3]:
# Configuración de Interactive Brokers
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)


<IB connected to 127.0.0.1:7497 clientId=1>

In [4]:
# Obtener datos de 90 días con velas de 1 hora
contract = Forex('GBPUSD')
bars = ib.reqHistoricalData(
    contract,
    endDateTime='',
    durationStr='250 D',
    barSizeSetting='1 hour',
    whatToShow='MIDPOINT',
    useRTH=True
)

In [5]:
# Convertir datos a DataFrame
df = pd.DataFrame(bars, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
df.set_index('date', inplace=True)

In [6]:
# Configuración de News API
NEWS_API_KEY = "b35c56d955ee45178c703f7f79c1dfca"

In [7]:
# Obtener noticias relacionadas con tasas de interés y política económica
url_news = f"https://newsapi.org/v2/everything?q=interest%20rate%20OR%20monetary%20policy&language=en&apiKey={NEWS_API_KEY}"
response_news = requests.get(url_news).json()
news_factor = len(response_news.get("articles", []))  # Cantidad de noticias encontradas


In [8]:
# Intentar obtener eventos económicos desde NewsAPI como alternativa
economic_news_url = f"https://newsapi.org/v2/everything?q=economic%20events%20OR%20GDP%20OR%20inflation%20OR%20unemployment&language=en&apiKey={NEWS_API_KEY}"
response_economic_news = requests.get(economic_news_url).json()
economic_events_factor = len(response_economic_news.get("articles", []))  # Cantidad de noticias económicas

In [9]:
# Aplicar simulación de Monte Carlo para medias móviles
simulations = 1000
steps = 10
simulated_averages = []
for _ in range(simulations):
    simulated_path = [df['close'].values[-1]]
    for _ in range(steps - 1):
        simulated_path.append(simulated_path[-1] * (1 + random.gauss(0, 0.01)))
    simulated_averages.append(np.mean(simulated_path))
ma_monte_carlo = np.mean(simulated_averages)


In [10]:
# Normalización de los datos
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df[['open', 'high', 'low', 'close']])

In [11]:
# Preparar datos para LSTM
sequence_length = 24  # Usamos 24 horas de datos previos
X, y = [], []
for i in range(len(data_scaled) - sequence_length):
    X.append(data_scaled[i:i + sequence_length])
    y.append(data_scaled[i + sequence_length])
X, y = np.array(X), np.array(y)

In [12]:
# Construcción del modelo LSTM
model = Sequential([
    Input(shape=(sequence_length, 4)),
    LSTM(100, return_sequences=True),
    Dropout(0.3),
    LSTM(100, return_sequences=False),
    Dropout(0.3),
    Dense(4, activation='linear')
])

model.compile(optimizer='adam', loss='mean_squared_error')


In [13]:
# Entrenar el modelo
model.fit(X, y, epochs=30, batch_size=32, validation_split=0.1)

# Obtener datos más recientes para la predicción
latest_data = data_scaled[-sequence_length:].reshape(1, sequence_length, 4)
predictions = []
end_time = datetime.now().replace(hour=21, minute=0, second=0)

Epoch 1/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - loss: 0.0344 - val_loss: 3.5058e-04
Epoch 2/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0048 - val_loss: 5.4644e-04
Epoch 3/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0037 - val_loss: 8.6539e-04
Epoch 4/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0030 - val_loss: 6.9719e-04
Epoch 5/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0026 - val_loss: 2.5748e-04
Epoch 6/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0023 - val_loss: 2.6687e-04
Epoch 7/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0020 - val_loss: 3.0279e-04
Epoch 8/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0016 - val_loss: 2.3294e-04
Epoch 9/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0016 - val_loss: 3.1970e-04
Epoch 10/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0015 - val_loss: 2.1806e-04
Epoch 11/30
168/168 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0013 - val_loss: 3.0923e-

In [14]:
for i in range(10):  # Predicción de 10 horas
    pred = model.predict(latest_data)
    pred_adjusted = pred + (news_factor * 0.0001) + (economic_events_factor * 0.0001) + (ma_monte_carlo * 0.00005)
    predictions.append(scaler.inverse_transform(pred_adjusted)[0])
    latest_data = np.append(latest_data[:, 1:, :], pred_adjusted.reshape(1, 1, 4), axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [15]:
# Crear un DataFrame con las predicciones
prediction_times = [end_time + timedelta(hours=i) for i in range(1, 11)]
predicted_df = pd.DataFrame(predictions, columns=['open', 'high', 'low', 'close'], index=prediction_times)

print(predicted_df)

                                open      high       low     close
2025-03-12 22:00:00.249924  1.298958  1.299414  1.298301  1.298827
2025-03-12 23:00:00.249924  1.299701  1.300148  1.299045  1.299546
2025-03-13 00:00:00.249924  1.300656  1.301095  1.300004  1.300480
2025-03-13 01:00:00.249924  1.301642  1.302076  1.300996  1.301454
2025-03-13 02:00:00.249924  1.302593  1.303024  1.301954  1.302399
2025-03-13 03:00:00.249924  1.303498  1.303928  1.302864  1.303303
2025-03-13 04:00:00.249924  1.304368  1.304797  1.303738  1.304174
2025-03-13 05:00:00.249924  1.305214  1.305644  1.304587  1.305022
2025-03-13 06:00:00.249924  1.306045  1.306476  1.305421  1.305855
2025-03-13 07:00:00.249924  1.306865  1.307297  1.306242  1.306677


In [16]:
ib.disconnect()